In [ ]:
#Fundamentos em Big Data
#Tarefa 04 Elastic Search
#Grupo 03
#11/02/2021

In [ ]:
#!pip install elasticsearch
#!pip install csv2es
#!pip install requests

In [ ]:
#Comandos Uteis no Terminal
#pip install -U elasticsearch
#sudo -i service elasticsearch status
#sudo -i service elasticsearch start
#sudo -i service elasticsearch restart

In [2]:
from elasticsearch import helpers, Elasticsearch
import csv
import csv2es

In [3]:
es = Elasticsearch()

In [4]:
df = spark.read.csv('DNPBA2017.csv', header=True)

In [5]:
#df = df.select(['NUMERODN', 'CODESTAB', 'IDADEMAE', 'ESCMAEAGR1']).limit(20).toPandas()
df = df.toPandas()

In [6]:
df.to_csv('DNPBA2017_es.csv', index=False)

### indexng

In [4]:
with open('DNPBA2017_es.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='my-index3', doc_type='my-type')
    
# ler indices no linux com: curl 'localhost:9200/_cat/indices?v'

/home/bigdata/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [26]:
%%time 
#Gere um subset com todas as mães que tem mais que 40 anos.


content = {
    "query": {
        "range": {
            "IDADEMAE": {
                "gt": 40
            }
        }
    }
}
res = es.search(index="my-index3", body=content)
res

CPU times: user 3.65 ms, sys: 0 ns, total: 3.65 ms
Wall time: 17 ms


{'took': 10,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3735, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'my-index3',
    '_type': 'my-type',
    '_id': '1iXZdncB-Jg2IKHd8jQb',
    '_score': 1.0,
    '_source': {'_c0': '2457',
     'NUMERODN': '72268345',
     'CODINST': 'MBA2900700001',
     'ORIGEM': '1',
     'NUMERODV': '8',
     'PREFIXODN': '30',
     'CODESTAB': '2487411',
     'CODMUNNASC': '290070',
     'LOCNASC': '1',
     'IDADEMAE': '49',
     'ESTCIVMAE': '1',
     'ESCMAE': '4',
     'CODOCUPMAE': '612005',
     'QTDFILVIVO': '00',
     'QTDFILMORT': '00',
     'CODMUNRES': '290070',
     'GESTACAO': '5',
     'GRAVIDEZ': '1',
     'PARTO': '1',
     'CONSULTAS': '4',
     'DTNASC': '23012017',
     'HORANASC': '2144',
     'SEXO': '1',
     'APGAR1': '07',
     'APGAR5': '09',
     'RACACOR': '4',
     'PESO': '3070',
     'IDANOMAL': '2',
     'DTCADASTRO': '01022017',
     '

In [86]:
PUT /nxapi/_mapping/'localhost:9200/my-index'?update_all_types
{
"properties": {
"<text_field_name>": {
"type": "text",
"fielddata": true
}
}
}

SyntaxError: invalid syntax (<ipython-input-86-3c8238be13d7>, line 1)

In [4]:
#Obtenha estatísticas descritivas do atributo de escolaridade da mãe (min, max, avg, std)
content = {
  "aggs": {
    "dnpba_stats": { "stats": { "field": "IDADEMAE" } }
  }
}

 #my_field = field.Text(fields={'raw': field.Keyword()})

In [5]:
res = es.search(index="my-index3",body=content)

RequestError: RequestError(400, 'search_phase_execution_exception', 'Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field instead. Alternatively, set fielddata=true on [IDADEMAE] in order to load field data by uninverting the inverted index. Note that this can use significant memory.')

In [ ]:
#Faça uma agregação para saber a quantidade de crianças nascidas em cada município da Bahia.

In [ ]:
#Gere um subset com todas as mães entre 20 e 50 anos que tiveram filhos no primeiro dia do ano.

### buscando termo

In [7]:
content = {
    "query": {
        "term": {
            "CODESTAB": "2786095"
         }
     },
}

In [8]:
res = es.search(index="my-index3", body=content)

In [ ]:
res

### buscando um intervalo

In [9]:
content = {
    "query": {
        "range": {
            "IDADEMAE": {
                "gte": 20,
                "lt": 30
            }
        }
    }
}

In [10]:
res = es.search(index="my-index", body=content)

In [ ]:
res

### busca lógica

In [75]:
content = {
    "query": {
        "bool": {
            "must": [
                {
                    "term": {
                        "IDADEMAE": 21
                    }
                },
                {
                    "term": {
                        "ESCMAEAGR1": "06"
                    }
                }
            ]
        }
    }
}

In [76]:
res = es.search(index="my-index", body=content)

In [ ]:
res

### searching methods

In [46]:
def buscaExata(numerodn, codestab, idademae, escmaeagr1, startId=0): 
    
    global es
    
    content = {
        'size': 30,
        'query': {
            'bool': {
                'must': [
                    {'match_phrase': {'NUMERODN': '"' + numerodn + '"'}},
                    {'match_phrase': {'CODESTAB': '"' + codestab + '"'}}, 
                    {'match': {'IDADEMAE': idademae}},
                    {'match': {'ESCMAEAGR1': escmaeagr1}}
                ]
            }
        }
    }
    force = True
    while force:
        try:
            res = es.search(index="my-index", body=content)
            force = False
        except:
            pass
    return res['hits']['hits']

In [52]:
def buscaAproximada(numerodn, codestab, idademae, escmaeagr1, startId=0):
    
    global es
    
    content = {
        'size': 100,
        'query': {
            'bool': {
                'should': [
                    {'match': {'NUMERODN': {'query': numerodn, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'CODESTAB': {'query': codestab, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'IDADEMAE': {'query': idademae, 'fuzziness':'AUTO', 'operator':'or', 'boost':'0.5'}}},
                    {'match': {'ESCMAEAGR1': {'query': escmaeagr1, 'fuzziness':'AUTO', 'operator':'or'}}}
                ]
            }
        }
    }
    force = True
    while force:
        try:
            res = es.search(index="my-index", body=content)
            force = False
        except:
            pass
    return res['hits']['hits']

In [53]:
buscaExata("72390242", "2786095", "25", "01")

[{u'_id': u'H8atyW8BbundlQk4fUZA',
  u'_index': u'my-index',
  u'_score': 10.55623,
  u'_source': {u'CODESTAB': u'2786095',
   u'ESCMAEAGR1': u'01',
   u'IDADEMAE': u'25',
   u'NUMERODN': u'72390242'},
  u'_type': u'my-type'}]

In [63]:
# um campo modificado (note que o retorno é vazio)
buscaExata("72390242", "2786095", "33", "01")

[]

In [57]:
buscaAproximada("72390242", "2786095", "25", "01")

[{u'_id': u'H8atyW8BbundlQk4fUZA',
  u'_index': u'my-index',
  u'_score': 9.236701,
  u'_source': {u'CODESTAB': u'2786095',
   u'ESCMAEAGR1': u'01',
   u'IDADEMAE': u'25',
   u'NUMERODN': u'72390242'},
  u'_type': u'my-type'},
 {u'_id': u'I8atyW8BbundlQk4fUZB',
  u'_index': u'my-index',
  u'_score': 5.278115,
  u'_source': {u'CODESTAB': u'7373120',
   u'ESCMAEAGR1': u'12',
   u'IDADEMAE': u'34',
   u'NUMERODN': u'72392109'},
  u'_type': u'my-type'}]

In [65]:
# # um campo modificado (mais de um candidato é retornado)
buscaAproximada("72390242", "2786095", "33", "01")

[{u'_id': u'H8atyW8BbundlQk4fUZA',
  u'_index': u'my-index',
  u'_score': 13.195287,
  u'_source': {u'CODESTAB': u'2786095',
   u'ESCMAEAGR1': u'01',
   u'IDADEMAE': u'25',
   u'NUMERODN': u'72390242'},
  u'_type': u'my-type'},
 {u'_id': u'KMatyW8BbundlQk4fUZB',
  u'_index': u'my-index',
  u'_score': 1.0641159,
  u'_source': {u'CODESTAB': u'2777770',
   u'ESCMAEAGR1': u'08',
   u'IDADEMAE': u'33',
   u'NUMERODN': u'69726416'},
  u'_type': u'my-type'},
 {u'_id': u'KcatyW8BbundlQk4fUZB',
  u'_index': u'my-index',
  u'_score': 1.0641159,
  u'_source': {u'CODESTAB': u'2777770',
   u'ESCMAEAGR1': u'06',
   u'IDADEMAE': u'33',
   u'NUMERODN': u'69726448'},
  u'_type': u'my-type'}]

In [62]:
print len(buscaAproximada("72390242", "2786095", "33", "01"))

3
